# Training Data Debug

In [2]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy import spatial
from matplotlib import cm

import torch
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from tqdm.auto import tqdm

from sphere import Sphere
from visualize import Visualize
from dh_grid import DHGrid
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Load the previous (old) data format

In [3]:
export_ds = '/media/berlukas/T7 Touch/data/nuscenes/processed/old'

cloud_filename = f"{export_ds}/clouds1.npy"
sem_clouds_filename = f"{export_ds}/new_sem_classes_gt1.npy"

print(f"Loading clouds from {cloud_filename}")
cloud_features_old = np.load(cloud_filename)
sem_cloud_features_old = np.load(sem_clouds_filename)
print(f"Shape of clouds is {cloud_features_old.shape} and sem clouds is {sem_cloud_features_old.shape}")

Loading clouds from /media/berlukas/T7 Touch/data/nuscenes/processed/old/clouds1.npy
Shape of clouds is (11230, 2, 200, 200) and sem clouds is (11230, 200, 200)


# Load the current data format

In [4]:
export_ds = '/mnt/data/datasets/nuscenes/processed'

cloud_filename = f"{export_ds}/clouds_fixed_sem.npy"

print(f"Loading clouds from {cloud_filename}")
cloud_features = np.load(cloud_filename)
sem_cloud_features = cloud_features[:, 2, :, :]
cloud_features = cloud_features[:, 0:2, :, :]

print(f"Shape of clouds is {cloud_features.shape}")
# print(f"Shape of clouds is {cloud_features.shape} and sem clouds is {sem_cloud_features.shape}")

Loading clouds from /mnt/data/datasets/nuscenes/processed/clouds_fixed_sem.npy
Shape of clouds is (11230, 2, 200, 200)


## Compare the two datasets

In [5]:
def mapIntensityToRGB(i):
    mask = np.where(i < 0)    
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    colors[mask] = 0
    return colors

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)        
    o3d.visualization.draw_geometries([pcd])
    
def visualize_sphere(cloud, ndim = 1, bw = 100):    
    _, points = DHGrid.CreateGrid(bw)
    cur_sem_cloud = np.reshape(cloud, (ndim, -1)).T
    points_xyz = DHGrid.ConvertPointsToEuclidean(points)        
    points_xyzi = np.column_stack((points_xyz, cur_sem_cloud[:,0]))
    visualize_pointcloud(points_xyzi)
    
def writeRawPointCloud(cloud, filename):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    colors = mapIntensityToRGB(cloud[:, 3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.io.write_point_cloud(filename, pcd)
    

In [8]:
visualize_sphere(sem_cloud_features_old[0,:,:], 1)
visualize_sphere(sem_cloud_features[0,:,:], 1, 100)

In [6]:
old_classes = np.unique(sem_cloud_features_old[0,:,:])
new_classes = np.unique(sem_cloud_features[0,:,:])
print(f'old classes are {old_classes}')
print(f'new classes are {new_classes}')

old classes are [0. 1. 2. 3. 4. 5. 7. 8.]
new classes are [1. 2. 3. 4. 5. 7. 8.]


In [7]:
np.array_equal(sem_cloud_features_old[0:10,:,:], sem_cloud_features[0:10,:,:])

False

In [17]:
np.array_equal(cloud_features_old[1,0,:,:], cloud_features[1,0,:,:])

False

In [18]:
visualize_sphere(cloud_features_old[1,0,:,:], 1, 100)
visualize_sphere(cloud_features[1,0,:,:], 1, 100)

In [22]:
diff = cloud_features_old[1,0,:,:] - cloud_features[1,0,:,:]
# visualize_sphere(diff)
visualize_sphere(cloud_features[1,0,:,:] + (diff*100000))